# Bitcoin Options Volatility Surface Fitting

This notebook demonstrates the volatility surface fitting process using both Traditional Wing Model and Time-Adjusted Wing Model.

## Overview
- **Traditional Wing Model**: Standard parameterized volatility surface
- **Time-Adjusted Wing Model**: Enhanced model with time-dependent adjustments
- **Model Comparison**: Side-by-side analysis of fitting performance

## Key Features
- Automated model selection based on RMSE
- Interactive volatility surface visualization
- Greeks calculation (Delta, Gamma, Vega)
- Range analysis for moneyness and strike boundaries

## 1. Setup and Configuration

In [1]:
# Import required libraries
import sys
import os
import numpy as np
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project setup
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print("✅ Environment setup complete")

📁 Current directory: /home/user/Python/Baseoffset-Fitting-Manager/notebooks
📁 Project root: /home/user/Python/Baseoffset-Fitting-Manager
✅ Environment setup complete


In [2]:
# Import project modules
from utils.market_data.orderbook_deribit_md_manager import OrderbookDeribitMDManager
from utils.volatility_fitter.wing_model.wing_model import WingModel
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel

print("✅ All modules imported successfully")

✅ All modules imported successfully


In [17]:
# Create configuration dictionary
config = {
    'date': '20240229',
    'option_expiry': '15MAR24',#'15MAR24',
    'snapshot_time': '2024-02-29T20:12:00',
    'snapshot_tolerance_seconds': 30    
}

print("📋 Configuration created:")
print(f"  - Expiry: {config['option_expiry']}")
print(f"  - Date: {config['date']}")
print(f"  - Snapshot time: {config['snapshot_time']}")

📋 Configuration created:
  - Expiry: 15MAR24
  - Date: 20240229
  - Snapshot time: 2024-02-29T20:12:00


## 2. Data Loading and Preparation

In [18]:
# Load market data
from utils.volatility_fitter.processed_data_loader import create_snapshot_option_chain, load_baseoffset_results, load_option_market_data
from utils.volatility_fitter.volatility_calculator import get_option_chains

date_str = config['date']
snapshot_time = datetime.strptime(config['snapshot_time'], "%Y-%m-%dT%H:%M:%S")
df_baseoffset = load_baseoffset_results(date_str)
df_option_md = load_option_market_data(date_str)

df_snapshot_md = create_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)
my_expiry = config['option_expiry']

df_option_chain = get_option_chains(df_snapshot_md, my_expiry, snapshot_time)

print(f"📊 Loaded {len(df_option_chain)} option contracts")
print(f"📅 Expiry: {my_expiry} Time: {snapshot_time}")
df_option_chain.head()

Available expiries: ['15MAR24', '28JUN24', '8MAR24', '3MAR24', '22MAR24', '1MAR24', '29FEB24', '31MAY24', '2MAR24', '27DEC24', '29MAR24', '26APR24', '27SEP24']
📊 Loaded 27 option contracts
📅 Expiry: 15MAR24 Time: 2024-02-29 20:12:00


timestamp,bid_price,ask_price,strike,bid_size,ask_size,bid_price_P,ask_price_P,S,bid_price_fut,ask_price_fut,expiry,tau,bid_size_P,ask_size_P,future_basis
datetime[ns],f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2024-02-29 20:12:00,0.283,0.3095,44000,6.4,5.0,0.0012,0.0016,61924.86,62416.0,62416.0,"""15MAR24""",0.039703,27.3,11.0,491.14
2024-02-29 20:12:00,0.2785,0.282,45000,2.0,5.0,0.0015,0.0019,61924.86,62416.0,62416.0,"""15MAR24""",0.039703,3.1,10.4,491.14
2024-02-29 20:12:00,0.263,0.2665,46000,1.5,5.0,0.0017,0.0022,61924.86,62416.0,62416.0,"""15MAR24""",0.039703,59.4,12.0,491.14
2024-02-29 20:12:00,0.2475,0.2505,47000,1.0,5.0,0.0021,0.0026,61924.86,62416.0,62416.0,"""15MAR24""",0.039703,56.9,12.6,491.14
2024-02-29 20:12:00,0.2315,0.235,48000,2.0,5.0,0.0026,0.0031,61924.86,62416.0,62416.0,"""15MAR24""",0.039703,28.3,8.0,491.14


In [19]:
from utils.pricer.option_constraints import tighten_option_spread
from utils.reporting.html_table_generator import generate_price_comparison_table
from IPython.display import HTML, display

tightened_option_chain = tighten_option_spread(df_option_chain)
print(f"Comparison of old bid/ask proces and the tightened bid/ask price")

# Generate HTML table using external module
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px")))

Comparison of old bid/ask proces and the tightened bid/ask price


In [20]:
# Get forward price and time to expiry
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks

df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=0.15)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)

assert len(df_option_with_vola['F'].unique().to_list()) == 1
assert len(df_option_with_vola['tau'].unique().to_list()) == 1
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]

print(f"💰 Forward price: ${forward_price:,.2f}")
print(f"⏰ Time to expiry: {time_to_expiry:.4f} years")


print("✅ DataFrame with vega calculations created successfully using centralized module!")
df_option_with_vola_and_greeks.select(['strike', 'bidVola', 'midVola', 'askVola', 'vega'])

💰 Forward price: $62,416.00
⏰ Time to expiry: 0.0397 years
✅ DataFrame with vega calculations created successfully using centralized module!


strike,bidVola,midVola,askVola,vega
i64,f64,f64,f64,f64
44000,87.23,89.3,91.37,5.98
45000,85.36,87.11,88.86,7.06
46000,82.13,84.06,85.99,8.02
47000,80.16,81.81,83.46,9.38
48000,78.27,79.68,81.09,10.98
…,…,…,…,…
66000,68.59,69.26,69.92,46.62
67000,68.97,69.66,70.35,44.75
68000,69.26,69.99,70.72,42.52


In [21]:
fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Model Initialization and Calibration

In [22]:
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model_calibrator import TimeAdjustedWingModelCalibrator

my_calibrator = TimeAdjustedWingModelCalibrator(use_norm_term=True)

strikes = df_option_with_vola_and_greeks['strike'].to_list()
mid_vola = (df_option_with_vola_and_greeks['midVola']/100).to_list() 
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()
weights = (df_option_with_vola_and_greeks['vega'] / max(market_vegas)).to_list()

# fitter_result =\
my_calibrator.calibrate(strike_list=strikes, market_vol_list=mid_vola, market_vega_list=market_vegas, weight_list=weights, forward_price=forward_price, time_to_expiry=time_to_expiry)
# fitter_result

TimeAdjustedCalibrationResult(success=True, parameters=TimeAdjustedWingModelParameters(atm_vol=np.float64(0.6814893342707722), slope=np.float64(0.0850350163621685), call_curve=np.float64(0.27950452794757963), put_curve=np.float64(0.4889194052733569), up_cutoff=np.float64(1.0), down_cutoff=np.float64(-1.0), up_smoothing=np.float64(0.5), down_smoothing=np.float64(0.5), forward_price=62416.0, time_to_expiry=0.03970319634703196), error=np.float64(100000.04961012185), message='Optimization terminated successfully')

In [23]:
from scipy import optimize
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model_calibrator import TimeAdjustedCalibrationResult, create_time_adjusted_wing_model_from_result
# for i in range(5):
#     weights[i] = 1
args = (strikes, mid_vola, market_vegas, weights, forward_price, time_to_expiry, True)
initial_guess = [
            0.726,    # atm_vol
            0.12,                # slope
            0.251,                # curve_up
            0.37,                # curve_down
            0.5,                # cut_up
            -0.85,               # cut_dn
            0.5,                # mSmUp
            1.5                 # mSmDn
        ]
# initial_guess = my_optimized_paramter.get_fitted_vol_parameter()

result=\
optimize.minimize(
    fun=my_calibrator._loss_function,
    x0=initial_guess,
    args=args,
    method=my_calibrator.method,
    bounds=my_calibrator._get_parameter_bounds(),
    # tol=my_calibrator.tolerance
)
print(result)
my_optimized_paramter = create_time_adjusted_wing_model_from_result(result.x, forward_price, time_to_expiry)
fitter_result = TimeAdjustedCalibrationResult(success=result.success, parameters=my_optimized_paramter, error=result.fun, message=result.message if hasattr(result, 'message') else "")
print(f"🔄 Calibrating Time-Adjusted Wing Model. RMSE: {fitter_result.error:.6f}")

for param, value in zip(my_optimized_paramter.get_parameter_names(), my_optimized_paramter.get_fitted_vol_parameter()):
    print(f"{param:15}: {value:.5f}")

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.04961040883703004
           x: [ 6.815e-01  8.509e-02  2.791e-01  4.892e-01  5.000e-01
               -8.500e-01  5.000e-01  1.500e+00]
         nit: 11
         jac: [ 4.595e-02 -3.439e-03 -5.743e-04  1.184e-03  0.000e+00
                0.000e+00  0.000e+00  0.000e+00]
        nfev: 109
        njev: 11
 multipliers: []
🔄 Calibrating Time-Adjusted Wing Model. RMSE: 0.049610
atm_vol        : 0.68149
slope          : 0.08509
call_curve     : 0.27906
put_curve      : 0.48915
up_cutoff      : 0.50000
down_cutoff    : -0.85000
up_smoothing   : 0.50000
down_smoothing : 1.50000


## 4. Volatility Surface Analysis

In [24]:
# my_optimized_paramter.down_cutoff = -0.25
# my_optimized_paramter.down_smoothing = 1.5

In [25]:
my_model = TimeAdjustedWingModel(parameters=my_optimized_paramter)
# Generate fitted volatility surface
strikes = df_option_with_vola_and_greeks['strike']
mid_vola = df_option_with_vola_and_greeks['midVola']/100  # Convert from percentage to decimal
bid_vola = df_option_with_vola_and_greeks['bidVola']/100  # Convert from percentage to decimal
ask_vola = df_option_with_vola_and_greeks['askVola']/100  # Convert from percentage to decimal

# Get fitted volatilities
fitted_vols = []
if (use_extended_strikes:=True):
    extended_strikes = sorted(np.linspace(start=10000, stop=strikes[0], endpoint=False, num=10).tolist() +
                              strikes.to_list() + 
                              [strikes[-1]+(x+1)*2000 for x in range(20)])
else:
    extended_strikes = strikes
for strike in extended_strikes:
    # print(strike, my_model.calculate_moneyness(forward_price, strike, time_to_expiry, my_optimized_paramter.atm_vol))    
    fitted_vols.append(my_model.calculate_volatility_from_strike(strike))

print(f"📈 Generated volatility surface with {len(fitted_vols)} points")
print(f"   Strike range: ${min(extended_strikes):,.0f} - ${max(extended_strikes):,.0f}")
print(f"   Vol range: {min(fitted_vols):.1%} - {max(fitted_vols):.1%}")

📈 Generated volatility surface with 57 points
   Strike range: $10,000 - $112,000
   Vol range: 67.8% - 143.8%


In [26]:
# Create volatility smile plot
fig = go.Figure()

# Add market volatility with bid/ask error bars
fig.add_trace(go.Scatter(
    x=strikes, y=mid_vola, mode='markers', name='Market IV (Bid/Ask Range)',
    marker=dict(color='blue', symbol='circle', size=8),
    error_y=dict(type='data', symmetric=False, array=ask_vola-mid_vola, arrayminus=mid_vola-bid_vola,
                 visible=True, color='blue', thickness=2, width=3),
    opacity=0.8
))

# Add the fitted volatility curve
fig.add_trace(go.Scatter(
    x=extended_strikes, y=fitted_vols, mode='lines+markers',
    name=f'{(model_name:=my_model.__class__.__name__)} Fit',
    line=dict(color='black', width=3, dash='solid'),
    marker=dict(color='black', symbol='diamond', size=10),
    opacity=0.5
))
# Add vertical lines for reference
forward_price = df_option_with_vola_and_greeks['F'][0]    
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Add vertical lines for different ranges
strike_ranges = my_model.get_strike_ranges()
print(f"Different ranges on the curve: {strike_ranges}")
fig.add_vline(x=(_price:=strike_ranges['downSmoothing']), line=(line_dict:=dict(color='green', dash='dot', width=1)), 
              annotation_text=f"DSm:{_price:.0f}", annotation_position="bottom", annotation_font=(font_dict:=dict(size=10, color="green")),)
fig.add_vline(x=(_price:=strike_ranges['downCutOff']), line=line_dict,     annotation_text=f"DCO:{_price:.0f}", annotation_position="bottom", annotation_font=font_dict)
fig.add_vline(x=(_price:=strike_ranges['upCutOff']), line=line_dict, annotation_text=f"UCO:{_price:.0f}", annotation_position="bottom", annotation_font=font_dict)
fig.add_vline(x=(_price:=strike_ranges['upSmoothing']), line=line_dict,   annotation_text=f"USm:{_price:.0f}", annotation_position="bottom", annotation_font=font_dict)

param_names = my_optimized_paramter.get_parameter_names()
param_values = my_optimized_paramter.get_fitted_vol_parameter()
param_string = " | ".join([f"{name}={value:.3f}" for name, value in zip(param_names, param_values)])
ts = df_option_with_vola_and_greeks['timestamp'][0]
title = f"{ts}: {model_name} vs Market Data - {my_expiry} Expiry"
fig.update_layout(
    title=dict(
        text=f"{title}<br><span style='font-size:14px'>Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f} | RMSE: {fitter_result.error:.6f}<br> {param_string}</span>",
        x=0.5
    ),
    xaxis_title='Strike Price ($)',
    yaxis_title='Implied Volatility (%)',
    height=500
)
fig.update_xaxes(range=[strikes[0]*0.95, strikes[-1]*1.05])

fig.show()

Different ranges on the curve: {'downSmoothing': np.float64(23102.18126411087), 'downCutOff': np.float64(42173.65134807225), 'upCutOff': np.float64(79763.6826807874), 'upSmoothing': np.float64(89754.82170124608)}


In [13]:
from utils.pricer.black76_option_pricer import Black76OptionPricer

df_option_with_vola_and_greeks['strike']
df_option_with_vola_and_greeks =\
df_option_with_vola_and_greeks.with_columns([
    pl.struct(['strike']).map_elements(
        lambda row: my_model.calculate_volatility_from_strike(row['strike'])*100, return_dtype=pl.Float64
    ).round(2).alias('fitted_vol')
]).with_columns([
    # Calculate option prices and Greeks using fitted volatility
    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).call_price(), return_dtype=pl.Float64
    ).round(2).alias('TV_C'),
    
    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).put_price(), return_dtype=pl.Float64
    ).round(2).alias('TV_P'),
    
    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).call_delta(), return_dtype=pl.Float64
    ).round(2).alias('delta_C'),
    
    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).put_delta(), return_dtype=pl.Float64
    ).round(2).alias('delta_P'),

    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).vega()/100, return_dtype=pl.Float64
    ).round(2).alias('vega'),

    pl.struct(['F', 'strike', 'tau', 'r', 'fitted_vol']).map_elements(
        lambda row: Black76OptionPricer(F=row['F'], K=row['strike'], T=row['tau'], r=row['r'], sigma=row['fitted_vol']/100).gamma(), return_dtype=pl.Float64
    ).round(5).alias('gamma'),
])

In [14]:
df_option_with_vola_and_greeks

timestamp,bq0_C,bp0_C,bp0_C_usd,ap0_C_usd,ap0_C,aq0_C,strike,bq0_P,bp0_P,bp0_P_usd,ap0_P_usd,ap0_P,aq0_P,S,F,expiry,tau,r,bidVola_C,askVola_C,bidVola_P,askVola_P,bidVola,askVola,midVola,volSpread,vega,fitted_vol,TV_C,TV_P,delta_C,delta_P,gamma
datetime[ns],f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-02-29 20:12:00,6.4,0.312,19320.56,20304.64,0.327892,5.0,42000,7.5,0.0003,18.58,30.96,0.0005,7.9,61924.86,62229.04,"""8MAR24""",0.020525,0.15,2.36,148.749663,111.668798,118.899911,111.67,118.9,115.28,7.23,1.92,117.76,20195.57,28.71,0.99,-0.01,0.0
2024-02-29 20:12:00,5.0,0.2805,17369.92,18304.64,0.295594,5.0,44000,0.2,0.0005,30.96,43.35,0.0007,18.2,61924.86,62229.04,"""8MAR24""",0.020525,0.15,2.36,132.452305,106.229607,111.187094,106.23,111.19,108.71,4.96,2.45,108.33,18208.87,35.87,0.99,-0.01,0.0
2024-02-29 20:12:00,5.0,0.2645,16379.13,17304.64,0.279446,5.0,45000,3.4,0.0005,30.96,49.54,0.0008,17.4,61924.86,62229.04,"""8MAR24""",0.020525,0.15,2.36,124.567595,100.07213,106.819493,100.07,106.82,103.44,6.75,2.75,103.44,17215.59,39.51,0.99,-0.01,0.0
2024-02-29 20:12:00,5.0,0.2485,15388.33,16304.64,0.263297,6.3,46000,3.1,0.0006,37.15,55.73,0.0009,8.3,61924.86,62229.04,"""8MAR24""",0.020525,0.15,2.36,116.845755,96.390293,102.215326,96.39,102.22,99.3,5.83,3.11,98.78,16223.33,44.18,0.98,-0.01,0.0
2024-02-29 20:12:00,5.0,0.2335,14459.45,15304.64,0.247149,5.0,47000,4.0,0.0008,49.54,68.12,0.0011,12.2,61924.86,62229.04,"""8MAR24""",0.020525,0.15,2.36,109.277389,94.161844,98.899332,94.16,98.9,96.53,4.74,3.59,94.56,15233.18,50.96,0.98,-0.02,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-29 20:12:00,15.4,0.012,743.1,774.06,0.0125,1.4,68000,67.5,0.103,6378.26,6687.88,0.108,67.5,61924.86,62229.04,"""8MAR24""",0.020525,0.15,72.811977,74.005566,68.119442,80.002616,72.81,74.01,73.41,1.2,25.95,73.44,759.35,6512.57,0.21,-0.78,0.00004
2024-02-29 20:12:00,44.6,0.0075,464.44,526.36,0.0085,49.3,70000,67.4,0.1305,8081.19,8421.78,0.136,68.0,61924.86,62229.04,"""8MAR24""",0.020525,0.15,73.780292,76.771151,66.914015,83.458294,73.78,76.77,75.28,2.99,20.82,75.62,502.18,8249.25,0.15,-0.85,0.00003
2024-02-29 20:12:00,0.5,0.005,309.62,340.59,0.0055,19.8,72000,67.4,0.16,9907.98,10279.53,0.166,67.5,61924.86,62229.04,"""8MAR24""",0.020525,0.15,76.381181,78.314902,65.951348,89.216938,76.38,78.31,77.34,1.93,16.23,77.96,334.81,10075.73,0.11,-0.89,0.00003


In [15]:
# Comprehensive arbitrage analysis using new module
from utils.analysis.arbitrage_checker import analyze_arbitrage_comprehensive, format_arbitrage_report


strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
call_prices_list = df_option_with_vola_and_greeks['TV_C'].to_list()
put_prices_list = df_option_with_vola_and_greeks['TV_P'].to_list()
forward_price = df_option_with_vola_and_greeks['F'][0]
interest_rate = df_option_with_vola_and_greeks['r'][0]

# Perform comprehensive arbitrage analysis
arbitrage_results = analyze_arbitrage_comprehensive(
    strikes=strikes_list,
    call_prices=call_prices_list, 
    put_prices=put_prices_list,
    forward_price=forward_price,
    discount_factor=np.exp(-interest_rate*time_to_expiry),  # For futures
    butterfly_tolerance=0.1,
    parity_tolerance=0.1
)

# Display formatted report
print(format_arbitrage_report(arbitrage_results))

🔍 ARBITRAGE ANALYSIS REPORT: ✅ ARBITRAGE-FREE
📊 Summary: 41 strikes analyzed
   Strike range: 42000 - 76000
   Total violations: 0

🦋 Butterfly Arbitrage: 0 violations
📈 Price Monotonicity:
   Call prices (decreasing): ✅
   Put prices (increasing): ✅
⚖️ Call-Put Parity: 0 violations


In [16]:
from utils.analysis.arbitrage_checker import check_call_put_parity


check_call_put_parity(strikes_list, call_prices_list, put_prices_list, forward_price, discount_factor=np.exp(-interest_rate*time_to_expiry), tolerance=0.1)

[]